In [6]:
import pandas as pd
import random
import json
import csv
from tqdm import tqdm

In [7]:
df = pd.read_excel('../Excel_files/Full_Poem_Dataset_12-17.xlsx')
df = df.drop(columns=['Unnamed: 0'])
records = df.to_dict('records')
random.choice(records)

{'Text': 'Снился мне странный дом, где говорила: не уходи. Ты мне отец и мать, ты мне огонь в груди. Что тут не вынимать из меня самой. Только вот не пизди, много тут не пизди. Я говорю: вернись, но вроде не уходил. Только вернись, мой друг, с плачем проснусь во тьме, будто бы на твоем плече, будто бы с сердцем твоим я расстаюсь навек. Только вернись как мой, только приди как сам. Как я за нас боюсь, как я за нас боюсь, ссыкотно небесам. Новое кабаре в мире живых калек. Слава чёрным чулкам, победа высоким трусам, красной помаде в видео шорт. В мире, где ждут войну Я запрошу антр ну, будешь ли ты со мной, любишь ли ты меня до ледяного дна, любишь ли как Христа? Всё что я раньше мог, всё, чем я раньше жил, я создавал с тобой. Ныне же кровь сворачивается из жил, язык примерзает ко льду. Надо идти до Ада, значит, что я иду.\nБоженька, посмотри, как мы делаем это, как цирковые, какой это адский, невероятный труд. Да и Ты бы взглянул, как Правитель падшего мира, кому ужасен, а нам смешон, в 

In [8]:
len(records)

3222

### Toponyms

In [99]:
oblasts = '''Черниговская область
Черкасская область
Хмельницкая область
Херсонская область
Харьковская область
Тернопольская область
Сумская область
Севастополь
Ровненская область
Полтавская область
Одесская область
Николаевская область
Львовская область
Луганская область
Кировоградская область
Киевская область
Киев
Ивано-Франковская область
Запорожская область
Закарпатская область
Житомирская область
Донецкая область
Днепропетровская область
Волынская область
Винницкая область
Крым'''
tabNames = '''Черниговщина	Чернігівщина	Чернигов	Чернігів
Черкасщина	Черкащина	Черкассы	Черкаси
Хмельнитчина	Хмельниччина	Хмельницкий	Хмельницький
Херсонщина	Херсонщина	Херсон	Херсон
Харьковщина	Харківщина	Харьков	Харків
Тернопольщина	Тернопільщина	Тернополь	Тернопіль
Сумщина	Сумщина	Сумы	Суми
		Севастополь	
Ровненщина	Рівненщина	Рівне
Полтавщина	Полтавщина	Полтава	Полтава
Одесщина	Одещина	Одесса	Одеса
Николаевщина	Миколаївщина	Николаев	Миколаїв
Львовщина	Львівщина	Львов	Львів
Луганщина	Луганщина	Луганск	Луганськ
Кировоградщина	Кіровоградщина	Кропивницкий	Кропивницький
Киевщина	Київщина	Киев	Київ
Ивано-Франковщина	Івано-Франківщина	Ивано-Франковск	Івано-Франківськ
Запорожье	Запоріжжя	Запорожье	Запоріжжя
Закарпатье	Закарпаття	Ужгород	Ужгород
Житомирщина	Житомирщина	Житомир	Житомир
Донетчина	Донеччина	Донецк	Донецьк
Днепропетровщина	Дніпропетровщина	Днепр	Дніпро
Волынщина	Волинщина	Луцк	Луцьк
Виннитчина	Вінниччина	Винница	Вінниця'''

In [100]:
otherCities = '''Киев
Харьков
Одесса
Днепр
Донецк
Львов
Запорожье
Кривой Рог
Севастополь
Николаев
Мариуполь
Луганск
Винница
Макеевка
Симферополь
Чернигов
Полтава
Херсон
Хмельницкий
Черкассы
Черновцы
Житомир
Сумы
Горловка
Ивано-Франковск
Каменское
Тернополь
Кропивницкий
Луцк
Кременчуг
Белая Церковь
Керчь
Мелитополь
Краматорск
Ужгород
Бровары
Евпатория
Бердянск
Алчевск
Никополь
Славянск
Павлоград
Северодонецк
Каменец-Подольский
Лисичанск
Красный Луч
Енакиево
Александрия
Стаханов
Кадиевка
Константиновка'''.split('\n')

In [101]:
oblasts = [d.replace(' область','') for d in oblasts.split('\n')]
cityNames = []
centerNames = [[cityNames.append(t) for t in l.split('\t') if t != ''] for l in tabNames.split('\n')]

In [102]:
geoNames = otherCities + oblasts + cityNames + 'Донбас Винница, Буча, Ирпень, Киев, Мариуполь, Одесса, Крым, Луганск, Донецк, Днепр, Запорожье, Львов, Харьков, Херсон, Симферополь, чернигов, Черновцы, житомир, Полтава, Николаев, Гостомель, Краматорск'.split(', ')
geoNames = list(set(geoNames))

In [128]:
### how manyc ity names?
cities = [d.lower() for d in geoNames+ ['Донбасс']]
cityDict = dict()
allMentions = {'Before' : 0,'After' : 0}
for rec in tqdm(records):
    p = rec['Before or after']
    for t in rec['spacydoc']:
        if t.lemma_.lower() in cities:
            cityDict.setdefault(t.lemma_, {'Before' : 0, 'After' : 0})
            cityDict[t.lemma_][p] += 1
            allMentions[p] += 1
print(allMentions)
cityDict

100%|██████████████████████████████████████| 3222/3222 [00:07<00:00, 413.50it/s]

{'Before': 53, 'After': 460}


{'крым': {'Before': 16, 'After': 10},
 'донбасс': {'Before': 5, 'After': 14},
 'мариуполь': {'Before': 2, 'After': 63},
 'харьков': {'Before': 3, 'After': 59},
 'киев': {'Before': 4, 'After': 80},
 'буча': {'Before': 0, 'After': 64},
 'винница': {'Before': 1, 'After': 11},
 'одесса': {'Before': 10, 'After': 21},
 'запорожье': {'Before': 1, 'After': 3},
 'херсон': {'Before': 2, 'After': 18},
 'николаев': {'Before': 0, 'After': 3},
 'лисичанск': {'Before': 0, 'After': 2},
 'мелитополь': {'Before': 0, 'After': 2},
 'северодонецк': {'Before': 0, 'After': 3},
 'полтава': {'Before': 0, 'After': 3},
 'донецк': {'Before': 3, 'After': 4},
 'днепр': {'Before': 3, 'After': 19},
 'чернигов': {'Before': 0, 'After': 13},
 'львов': {'Before': 1, 'After': 9},
 'сумы': {'Before': 0, 'After': 2},
 'ирпень': {'Before': 0, 'After': 20},
 'гостомель': {'Before': 0, 'After': 6},
 'керчь': {'Before': 1, 'After': 0},
 'кременчуг': {'Before': 0, 'After': 2},
 'кропивницкий': {'Before': 0, 'After': 1},
 'славян

In [115]:
cityDict.keys()

dict_keys(['украина', 'крым', 'мариуполь', 'харьков', 'киев', 'буча', 'винница', 'одесса', 'запорожье', 'херсон', 'николаев', 'лисичанск', 'мелитополь', 'северодонецк', 'полтава', 'донецк', 'днепр', 'чернигов', 'львов', 'сумы', 'ирпень', 'гостомель', 'керчь', 'кременчуг', 'кропивницкий', 'славянск', 'черновцы', 'харків', 'київ', 'краматорск', 'луганск', 'бердянск', 'житомир', 'горловка', 'луцк', 'никополь', 'киевская', 'закарпатье'])

In [112]:
cs = '''Gostomel
Dnieper
Donetsk
Zhitomir
Transcarpathia
Zaporozhye
Irpin
Kerch
Kyiv/Kyiv/Kiev
Kramatorsk
Kremenchug
Kropyvnytskyi
Crimea
Lisichansk
Lugansk
Lutsk
Lviv
Mariupol
Melitopol
Nikolaev
Nikopol
Odessa
Poltava
Severodonetsk
Slavyansk
Sumy
Kharkiv/Kharkov
Kherson
Chernihiv
Chernivtsi'''.split('\n')
for c in cs:
    print(c)

Gostomel
Dnieper
Donetsk
Zhitomir
Transcarpathia
Zaporozhye
Irpin
Kerch
Kyiv/Kyiv/Kiev
Kramatorsk
Kremenchug
Kropyvnytskyi
Crimea
Lisichansk
Lugansk
Lutsk
Lviv
Mariupol
Melitopol
Nikolaev
Nikopol
Odessa
Poltava
Severodonetsk
Slavyansk
Sumy
Kharkiv/Kharkov
Kherson
Chernihiv
Chernivtsi


## Looking at most labels

In [135]:
import math
math.sqrt(4)

2.0

In [190]:
print(random.choice(networkJson['Russia']['links']))
# keyword = 'мариуполь'
            # value = math.sqrt(link['linkCtAfter']) - math.sqrt(link['linkCtBefore'])
            # lst.append({'value' : value, 'info' : link})
# linkedWords = set()
# linkUps = {'Before' : [], 'After' : []}
lst = []

city = 'надежда'
for where in networkJson:
    ct = 0
    for link in tqdm(networkJson[where]['links']):
        if city in [link['sourceLemma'],link['targetLemma']] and (link['sourceLemma'] not in stopwords) and (link['targetLemma'] not in stopwords):
            ct += 1
            # if (link['sourceLemma'] not in stopwords) and (link['targetLemma'] not in stopwords):
            # if (len(link['sourceLemma']) > 2) and (len(link['targetLemma']) > 2):
            value = link['linkCtAfter'] + link['linkCtBefore']
            lst.append({'value' : value, 'info' : link})
    print(where, ct)

IndexError: list index out of range

In [ ]:
for d in sorted(lst, key=lambda x: x['value'],reverse=True)[:20]:
    print(d['info']['sourceLemma'], d['info']['targetLemma']) 
    print(d['info']['linkCtBefore'],d['info']['linkCtAfter'])

In [174]:
# for d in sorted(lst, key=lambda x: (x['linkCtBefore'] + x['linkCtAfter']),reverse=True):
#     # if d['linkCtBefore'] + d['linkCtAfter'] > 4:
#     if d['linkCtAfter'] - d['linkCtBefore'] > 3:
#         if d['linkCtBefore'] < d['linkCtAfter']:
#             # print(d['linkCtAfter'])
#             print(d['sourceLemma'], d['targetLemma']) 
#             print(d['linkCtBefore'],d['linkCtAfter'])

KeyError: 'linkCtBefore'

In [279]:
# for d in sorted(linkUps['After'], key=lambda x: x['value'],reverse=True)[]:
#     print(d['info']['sourceLemma'], d['info']['targetLemma']) 
#     print(d['info']['linkCtBefore'],d['info']['linkCtAfter'])

## Checking specific labels

In [124]:
with open(f'../../label2lines_3-3.json', 'r', encoding='utf-8') as f:
    linedata = label2lines

In [125]:
random.choice(list(linedata.keys()))

'порождатьANDсуществовать'

### How often are two words used together?

In [ ]:
for rec in tqdm(records):
        
    if rec['Before or after'] != period:
        continue

    doc = rec['spacydoc']
    newdoc = [t for t in doc if (t.text.isalpha() or t.text == '\n')]
    doclen = len(newdoc)
    # # line jumping
    allLines = []
    newLine = []
    for i, t in enumerate(newdoc):

In [128]:
w1 = 'запад'
w2 = 'киев'
key ='AND'.join(sorted([w1,w2]))
for d in linedata[key]:
    print(d['period'], '\n', d['excerpt'],'\n')

KeyError: 'западANDкиев'

## nearest neighbors of cities

In [46]:
with open(f'../../files2big/before-after-average-BERT-PROPN_0.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

In [47]:
with open(f'../../files2big/before-after-average-BERT-1.json', 'r', encoding='utf-8') as f:
    newdata = json.load(f)
data.update(newdata)

In [74]:
# for place in t.split('\n'):
#     newdata[place] = data[place]

In [48]:
len(newdata), len(data)

(798, 1047)

In [77]:
# with open(f'../../files2big/all-BERT.json', 'w') as json_file:
#     json.dump(newdata, json_file, ensure_ascii = False, indent=4)

In [96]:
# with open('../averageTSNE/pnouns.txt','r') as f:
#     pns = f.read()

In [70]:
# with open(f'../averageTSNE/places.json', 'w') as json_file:
#     json.dump(t.split('\n'), json_file, ensure_ascii = False, indent=4)


In [49]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

def cosim(a,b):
    a = np.array(a)
    b = np.array(b)
    return dot(a, b)/(norm(a)*norm(b))

In [116]:
cities = ['крым', 'мариуполь', 'харьков', 'киев', 'буча', 'винница', 'одесса', 'запорожье', 'херсон', 'николаев', 'лисичанск', 'мелитополь', 'северодонецк', 'полтава', 'донецк', 'днепр', 'чернигов', 'львов', 'сумы', 'ирпень', 'гостомель', 'керчь', 'кременчуг', 'кропивницкий', 'славянск', 'черновцы', 'харків', 'київ', 'краматорск', 'луганск', 'бердянск', 'житомир', 'горловка', 'луцк', 'никополь', 'киевская', 'закарпатье']
random.choice(cities)

'бердянск'

In [126]:
keyword = 'буча'
periods = ['Average','Before','After']

cosinesims = dict()
for d in tqdm(data):
    for p in periods:
        cosinesims.setdefault(p, [])
        if not data[d][p]==np.zeros(768).tolist() and not data[keyword][p]==np.zeros(768).tolist():
            cosinesims[p].append({'cosim' : cosim(data[d][p],data[keyword][p]), 'word' : d})

100%|██████████████████████████████████████| 1047/1047 [00:01<00:00, 992.22it/s]


In [127]:
sorted(cosinesims['After'], key=lambda x: x['cosim'],reverse=True)[:10]

[{'cosim': 1.0, 'word': 'буча'},
 {'cosim': 0.9086308734617394, 'word': 'буче'},
 {'cosim': 0.8982766065857158, 'word': 'бучу'},
 {'cosim': 0.5940954032209228, 'word': 'буратино'},
 {'cosim': 0.5156366073879157, 'word': 'саша'},
 {'cosim': 0.508578160103435, 'word': 'луцк'},
 {'cosim': 0.48154723321623044, 'word': 'киев'},
 {'cosim': 0.4710904695285041, 'word': 'град'},
 {'cosim': 0.4703535255279665, 'word': 'ирпень'},
 {'cosim': 0.4700514796340463, 'word': 'польша'}]

In [200]:
[{'cosim': 1.0, 'word': 'ирпень'},
 {'cosim': 0.874249479937607, 'word': 'ирпене'},
 {'cosim': 0.5093010577205944, 'word': 'луцк'},
 {'cosim': 0.5038240634210285, 'word': 'чернигов'},
 {'cosim': 0.4933369825133888, 'word': 'чижик'},
 {'cosim': 0.48220810231405437, 'word': 'гостомель'},
 {'cosim': 0.47376990802902696, 'word': 'киев'},
 {'cosim': 0.4703535255279665, 'word': 'буча'},
 {'cosim': 0.4684321289865723, 'word': 'рязань'},
 {'cosim': 0.46359688429203494, 'word': 'и'}]

[{'cosim': 1.0, 'word': 'мариуполь'},
 {'cosim': 0.6187862678638423, 'word': 'донецк'},
 {'cosim': 0.5945884674497959, 'word': 'харьков'},
 {'cosim': 0.5671606218504281, 'word': 'чернигов'},
 {'cosim': 0.5318473334960235, 'word': 'луцк'},
 {'cosim': 0.5266546988120171, 'word': 'херсон'},
 {'cosim': 0.49026229317379005, 'word': 'киев'},
 {'cosim': 0.4542774967366994, 'word': 'мандельштам'},
 {'cosim': 0.4534011455908419, 'word': 'петрович'},
 {'cosim': 0.453060231378015, 'word': 'донбасс'}]

['бумага', 'разговор', 'зуб', 'поэт', 'народ', 'граница', 'сложный', 'песня', 'общий', 'страна', 'берег', 'сторона', 'корень', 'знак', 'звук', 'рука', 'ребёнок', 'предмет', 'лодка', 'слово', 'крик', 'говорить', 'речь', 'губа', 'корабль', 'враг', 'книга', 'стих', 'кожа', 'кость', 'лист', 'земля', 'русский', 'голос', 'взгляд', 'язык', 'дух']


## Word Co-occurrence

### Initialize Spacy models

In [9]:
import spacy

In [10]:
# !python -m spacy download uk_core_news_lg

In [11]:
# !python -m spacy download ru_core_news_lg

In [12]:
nlp = spacy.load("ru_core_news_lg", disable=['attribute_ruler','ner'])

### Generate Docs

In [13]:
for rec in tqdm(records):
    recTxt = rec['Text']
    # if not isinstance(recTxt,str):
    #     recTxt = str(recTxt)
    doc = nlp(recTxt)
    rec['spacydoc'] = doc

100%|██████████████████████████████| 3222/3222 [03:56<00:00, 13.65it/s]


### Run cooc parser

In [14]:
import itertools
import string

In [15]:
from spacy.lang.ru import stop_words
ru_stops = stop_words.STOP_WORDS

from spacy.lang.en import stop_words
en_stops = stop_words.STOP_WORDS

from spacy.lang.uk import stop_words
uk_stops = stop_words.STOP_WORDS

from spacy.lang.es import stop_words
es_stops = stop_words.STOP_WORDS

from spacy.lang.pl import stop_words
pl_stops = stop_words.STOP_WORDS

stopwords = list(ru_stops)+list(uk_stops)
foreign_stops = list(en_stops)+list(es_stops)+list(pl_stops)
short_stops = [s for s in stopwords if len(s) < 3]
short_ru_stops = [s for s in ru_stops if len(s) < 3]

In [16]:
def isLemma(txt):
    if txt in ['быть','бывать','будет','будем','буду','будут']:
        return True
    for p in string.punctuation+'—'+'–':
        if p in txt:
            return False
    if txt.isspace():
        return False
    # if txt in foreign_stops:
        return False
    # if txt in short_stops:
    #     return False
    if txt in stopwords:
        return False
    return True

In [17]:
isLemma('он')

False

In [18]:
isLemma('сказать')

True

## Before and after separate

In [19]:
for period in ['Before','After'][1:]:
    
    lens=set()
    lenCts = dict()

    networkJson = dict()
    networkJson['nodes'] = []
    networkJson['links'] = []

    curatedNodes = set()
    linkCounter = dict()
    lemmaCounter = dict()

    print('finding sufficient nodes...')

    for rec in tqdm(records):
        # if rec['Before or after'] != period:
            # continue
            
        recTxt = rec['Text']

        doc = rec['spacydoc']
        for t1 in doc:
            l1 = t1.lemma_.lower()
            lemmaCounter.setdefault(l1, 0)
            lemmaCounter[l1] += 1

    # find those nodes which occur enough times
    sufficientNodes = set()
    for lemma in lemmaCounter:
        if lemmaCounter[lemma] >= 5 and isLemma(lemma):
            sufficientNodes.add(lemma.lower())


    label2lines = dict()
    # label2recs = dict()
    label2authors = dict()

    print('parsing links...')
    for rec in tqdm(records):
        
        if rec['Before or after'] != period:
            continue
        
        doc = rec['spacydoc']
        newdoc = [t for t in doc if (t.text.isalpha() or t.text == '\n')]
        doclen = len(newdoc)
        # # line jumping
        allLines = []
        newLine = []
        for i, t in enumerate(newdoc):
            if t.text == '\n':
                allLines.append(newLine)
                newLine = []
            elif i == doclen-1:
                allLines.append(newLine)
            else:
                newLine.append(t)
        numLines = len(allLines)
        windowlength = 4

        labelFoundFromLine = dict()
        for i1 in range(numLines-windowlength):
            # all tokens in the desired range
            tokensInWindow = []
            excerptLines = []
            for line in allLines[i1:i1+windowlength]:
                excerptLines.append(' '.join([t.text for t in line]))
                for token in line:
                    tokensInWindow.append(token)
            tiw = len(tokensInWindow)
            lens.add(tiw)
            lenCts.setdefault(tiw,0)
            lenCts[tiw] += 1
            if tiw > 50:
                continue

            for ic, (t1, t2) in enumerate(itertools.combinations(tokensInWindow,2)):
                l1 = t1.lemma_.lower()
                l2 = t2.lemma_.lower()
                if l1 != l2 and l1 in sufficientNodes and l2 in sufficientNodes:
                    label = 'AND'.join(sorted([l1, l2]))

                    ### label already found in line?
                    repeatedInstance = False
                    for line in excerptLines:
                        labelFoundFromLine.setdefault((label, line), False)
                        if labelFoundFromLine[(label, line)]:
                            repeatedInstance = True
                        else:
                            labelFoundFromLine[(label, line)] = True
                    if repeatedInstance:
                        continue

                    label2authors.setdefault(label, set())
                    label2authors[label].add(rec['Author'])
#                     label2recs.setdefault(label, [])
#                     if rec not in label2recs[label]:
#                         label2recs[label].append(rec)

                    excerpt = '\n'.join(excerptLines)
                    label2lines.setdefault(label, [])
                    if excerpt not in [e['excerpt'] for e in label2lines[label]]:
                        # increase link Ct
                        linkCounter.setdefault(label, 0)
                        linkCounter[label] += 1
                        period = rec['Before or after']
                        if pd.isna(period):
                            period = ''
                        author = rec['Author']
                        if pd.isna(author):
                            author = 'Unknown'
                        label2lines[label].append({
                            'excerpt' : excerpt,
                            'author' : author,
                            'period' : period,
                            'uniqueIndex' : rec['UniqueIndex']
                        })

    node2id = dict()
    for i, node in enumerate(sufficientNodes):
        networkJson['nodes'].append({
            'id' : node,
            'totalinstances' : lemmaCounter[node]
        })
        node2id[node] = i

    for label in linkCounter:
        source = label.split('AND')[0]
        target = label.split('AND')[1]
        if lemmaCounter[source] > 9 and lemmaCounter[target] > 9:
            networkJson['links'].append({
                'source' : node2id[source],
                'target' : node2id[target],
                'sourceLemma' : source,
                'targetLemma' : target,
                'linkCt' : linkCounter[label],
                'authorCt' : list(label2authors[label])
            })

    # with open(f'../wordnet/data/{period}-wordnet2lines_allstops_2-21.json', 'w') as json_file:
    #     json.dump(networkJson, json_file, ensure_ascii = False, indent=4)
    # with open(f'../wordnet/data/{period}-label2lines_2-21.json', 'w') as json_file:
    #     json.dump(label2lines, json_file, ensure_ascii=False, indent=4)
    print('wrote the jsons!')

finding sufficient nodes...


100%|█████████████████████████████| 3222/3222 [00:07<00:00, 458.20it/s]


parsing links...


100%|█████████████████████████████| 3222/3222 [00:18<00:00, 172.20it/s]


wrote the jsons!


In [21]:
random.choice(networkJson['links'])

{'source': 6397,
 'target': 1554,
 'sourceLemma': 'воздух',
 'targetLemma': 'тьма',
 'linkCt': 2,
 'authorCt': ['Михаил Айзенберг', 'Геннадий Каневский']}

In [22]:
w1 = 'запад'
w2 = 'киев'
key ='AND'.join(sorted([w1,w2]))
for d in label2lines[key]:
    print(d['period'], '\n', d['excerpt'],'\n')

KeyError: 'западANDкиев'

## Before and After Together

In [13]:
authorsDf = pd.read_excel('../Excel_files/Thesis Authors.xlsx')
authorsDf = authorsDf.fillna('N/A')
aRecs = authorsDf.to_dict('records')
authorDict = dict()
for a in aRecs:
    authorDict[a['Author']] = a
authorsDf.head()

,Number of Poems,Author,City,Country,Identification,Notes,Link to bio
0,3,Виген Аракелян,Razdan,Armenia,N/A,N/A,https://polutona.ru/?show=arakelian
1,1,Герман Лукомников,Baku,Azerbaijan,N/A,N/A,https://ru.wikipedia.org/wiki/%D0%9B%D1%83%D0%...
2,1,Александр Иличевский,Sumgait,Azerbaijan,N/A,N/A,https://ru.wikipedia.org/wiki/%D0%98%D0%BB%D0%...
3,7,Леонид Шваб,Babruysk,Belarus,N/A,N/A,http://www.litkarta.ru/world/israel/persons/sh...
4,6,Ирина Валерина,Babruysk,Belarus,N/A,N/A,N/A


In [22]:
authorDict[rec['Author']]['Country']

'N/A'

In [23]:
lens=set()
lenCts = dict()

# networkJson = {'Non-Russia' : dict(), 'Russia' : dict(), 'Unknown' : dict()}
# for aff in networkJson:
for aff in ['all']:
    networkJson.setdefault(aff, dict())
    networkJson[aff]['nodes'] = []
    networkJson[aff]['links'] = []

lemmaCounter = dict()

print('finding sufficient nodes...')

for rec in tqdm(records):
    # if rec['Before or after'] != period:
        # continue

    recTxt = rec['Text']

    doc = rec['spacydoc']
    for t1 in doc:
        l1 = t1.lemma_.lower()
        lemmaCounter.setdefault(l1, 0)
        lemmaCounter[l1] += 1

# find those nodes which occur enough times
sufficientNodes = set()
for lemma in lemmaCounter:
    if lemmaCounter[lemma] >= 5 and isLemma(lemma):
        sufficientNodes.add(lemma.lower())

print('parsing links...')
for aff in ['all']:
    print(aff)
    
    allLabels = set()
    label2lines = dict()
    # label2recs = dict()
    label2authors = dict()
    linkCounter = {'Before' : dict(), 'After' : dict()}
    
    for rec in tqdm(records):
        
#         if rec['Author'] not in authorDict:
#             continue
#         if authorDict[rec['Author']]['Country'] == 'Russia':
#             thisaff = 'Russia'
#         elif authorDict[rec['Author']]['Country'] == 'N/A':
#             thisaff = 'Unknown'
#         else:
#             thisaff = 'Non-Russia'
            
#         if thisaff != aff:
#             continue
        
        period = rec['Before or after']
        doc = rec['spacydoc']
        newdoc = [t for t in doc if (t.text.isalpha() or t.text == '\n')]
        doclen = len(newdoc)
        # # line jumping
        allLines = []
        newLine = []
        for i, t in enumerate(newdoc):
            if t.text == '\n':
                allLines.append(newLine)
                newLine = []
            elif i == doclen-1:
                allLines.append(newLine)
            else:
                newLine.append(t)
        numLines = len(allLines)
        windowlength = 2

        labelFoundFromLine = dict()
        for i1 in range(numLines-windowlength):
            # all tokens in the desired range
            tokensInWindow = []
            excerptLines = []
            for line in allLines[i1:i1+windowlength]:
                excerptLines.append(' '.join([t.text for t in line]))
                for token in line:
                    tokensInWindow.append(token)
            tiw = len(tokensInWindow)
            lens.add(tiw)
            lenCts.setdefault(tiw,0)
            lenCts[tiw] += 1
            if tiw > 50:
                continue

            for ic, (t1, t2) in enumerate(itertools.combinations(tokensInWindow,2)):
                l1 = t1.lemma_.lower()
                l2 = t2.lemma_.lower()
                if l1 != l2 and l1 in sufficientNodes and l2 in sufficientNodes:
                    label = 'AND'.join(sorted([l1, l2]))

                    ### label already found in line?
                    repeatedInstance = False
                    for line in excerptLines:
                        labelFoundFromLine.setdefault((label, line), False)
                        if labelFoundFromLine[(label, line)]:
                            repeatedInstance = True
                        else:
                            labelFoundFromLine[(label, line)] = True
                    if repeatedInstance:
                        continue

                    label2authors.setdefault(label, {'Before' : set(), 'After' : set()})
                    label2authors[label][period].add(rec['Author'])


                    # label2recs.setdefault(label, [])
                    # if rec not in label2recs[label]:
                    #     label2recs[label].append(rec)

                    excerpt = '\n'.join(excerptLines)
                    label2lines.setdefault(label, [])
                    if excerpt not in [e['excerpt'] for e in label2lines[label]]:
                        # increase link Ct
                        for p in ['Before','After']:
                            linkCounter[p].setdefault(label, 0)
                        linkCounter[period][label] += 1
                        allLabels.add(label)

                        if pd.isna(period):
                            period = ''
                        author = rec['Author']
                        if pd.isna(author):
                            author = 'Unknown'
                        label2lines[label].append({
                            'excerpt' : excerpt,
                            'author' : author,
                            'period' : period,
                            'uniqueIndex' : rec['UniqueIndex']
                        })

    node2id = dict()
    for i, node in enumerate(sufficientNodes):
        networkJson[aff]['nodes'].append({
            'id' : node,
            'totalinstances' : lemmaCounter[node],
            'pos' : nlp(node)[0].pos_
        })
        node2id[node] = i

    for label in allLabels:
        source = label.split('AND')[0]
        target = label.split('AND')[1]
        if lemmaCounter[source] > 9 and lemmaCounter[target] > 9:
            label2authors[label]['Before'] = list(label2authors[label]['Before'])
            label2authors[label]['After'] = list(label2authors[label]['After'])

            networkJson[aff]['links'].append({
                'source' : node2id[source],
                'target' : node2id[target],
                'label' : label,
                'sourceLemma' : source,
                'targetLemma' : target,
                'linkCtBefore' : linkCounter['Before'][label],
                'linkCtAfter' : linkCounter['After'][label],
                'authors' : label2authors[label],
            })
    print(sorted([l['linkCtAfter'] for l in networkJson[aff]['links']],reverse=True)[:10])
    # print(len(networkJson['links']))
    # with open(f'../../allLinkData3-3_21.json', 'w') as json_file:
    #     json.dump(networkJson, json_file, ensure_ascii = False, indent=4)
    # with open(f'../../label2lines_3-21.json', 'w') as json_file:
    #     json.dump(label2lines, json_file, ensure_ascii=False, indent=4)
    print('wrote the json!')

finding sufficient nodes...


100%|████████████████████████████| 3222/3222 [00:01<00:00, 1904.91it/s]


parsing links...
all


100%|█████████████████████████████| 3222/3222 [00:17<00:00, 179.74it/s]


[47, 34, 33, 27, 26, 25, 24, 24, 23, 23]
wrote the json!


In [193]:
    # with open(f'../../allLinkData3-3_22.json', 'w') as json_file:
    #     json.dump(networkJson, json_file, ensure_ascii = False, indent=4)
    # with open(f'../../label2lines_3-22.json', 'w') as json_file:
    #     json.dump(label2lines, json_file, ensure_ascii=False, indent=4)

In [25]:
w1, w2 = ('язык','ты')
label = 'AND'.join(sorted([w1, w2]))
for who in ['all']:
    for l in networkJson[who]['links']:
        if l['label'] == label:
            print(who,l)
    print()

In [41]:
# Russia {'source': 7588, 'target': 7382, 'label': 'ониANDязык', 'sourceLemma': 'они', 'targetLemma': 'язык', 'linkCtBefore': 4, 'linkCtAfter': 1, 'authors': {'Before': ['Улад Клавкин', 'Игорь Булатовский', 'Андрей Черкасов', 'Екатерина Перченкова'], 'After': ['Юлия Пикалова']}}

# Non-Russia {'source': 7588, 'target': 7382, 'label': 'ониANDязык', 'sourceLemma': 'они', 'targetLemma': 'язык', 'linkCtBefore': 0, 'linkCtAfter': 3, 'authors': {'Before': [], 'After': ['Анна Стреминская', 'Таня Скарынкина', 'Вадим Гройсман']}}

# Unknown {'source': 7588, 'target': 7382, 'label': 'ониANDязык', 'sourceLemma': 'они', 'targetLemma': 'язык', 'linkCtBefore': 0, 'linkCtAfter': 1, 'authors': {'Before': [], 'After': ['Марина Охримовская']}}

### Connections, translations, counts

In [97]:
#  occur the most with the propns
counter = dict()
for pn in pns.split('\n'):
    for link in networkJson['links']:
        if pn == link['sourceLemma']:
            targetWord = link['targetLemma']
        elif pn == link['targetLemma']:
            targetWord = link['sourceLemma']
        else:
            targetWord = ''
        if targetWord != '':
            counter.setdefault(targetWord, 0)
            counter[targetWord] += link['linkCtAfter']
lst = []
for c in counter:
    lst.append({'ct' : counter[c], 'word' : c})
srted = sorted(lst, key=lambda x: x['ct'],reverse=True)

In [101]:
with open(f'../averageTSNE/sortedConnections.json', 'w') as json_file:
    json.dump(srted, json_file, ensure_ascii=False, indent=4)

In [102]:
import translators as ts

Using United States server backend.


In [103]:
ts.google('буча')

'Bucha'

In [110]:
translations = dict()
for k in tqdm(newdata):
    translations[k] = ts.google(k)

100%|█████████████████████████████████████████| 851/851 [20:28<00:00,  1.44s/it]


In [111]:
len(newdata), len(translations)

(851, 851)

In [113]:
translations['херсон']

'Kherson'

In [114]:
# with open(f'../averageTSNE/translationsBERT.json', 'w') as json_file:
#     json.dump(translations, json_file, ensure_ascii=False, indent=4)

In [120]:
# with open(f'../averageTSNE/lemmaCounter.json', 'w') as json_file:
#     json.dump(lemmaCounter, json_file, ensure_ascii=False, indent=4)

In [122]:
lemmaCounter['египет']

6

### Rossiya Budet 